In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown

# Load environment variables and configure client
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key = openai_api_key)

print("OpenAI client configured.")

OpenAI client configured.


In [2]:
# Define a function for printing markdown cells
def print_markdown(text):
    display(Markdown(text))

In [3]:
# Import required classes from the agents module
# Agent: Represents an AI agent with a specific role and instructions
# Runner: Executes the agent and handles interactions
# SQLiteSession: Provides persistent memory storage for the agent using SQLite

from agents import Agent, Runner, SQLiteSession

# Define the role and instructions for the AI agent
market_researcher_instructions = """
Context:
You are a market research assistant helping analyze companies, industries, and competitors.

Instructions:
When given a question, provide a short factual answer based on your knowledge.

Output:
Start with a verdict prefix: either "✅ FACT:" or "❌ UNKNOWN:"
Follow with a concise one-sentence explanation.
"""

# Create an instance of the Agent
market_researcher_agent = Agent(name = "Market Researcher",
                                instructions = market_researcher_instructions,
                                model = "gpt-4.1-mini")

In [4]:
# Example: An AI Agent with NO memory

# Let's give our first question to the AI agent
q1 = "What is the market share of Tesla in the US EV market?"

# Display the user’s question in Markdown format
print_markdown(f"You: '{q1}'")

# Run the agent with the first question (no memory means each query is independent)
resp1 = await Runner.run(starting_agent = market_researcher_agent, input = q1)

# Display the agent’s response
print_markdown(f"🤖 Agent:\n{resp1.final_output}")

You: 'What is the market share of Tesla in the US EV market?'

🤖 Agent:
✅ FACT: Tesla holds approximately 65-70% of the U.S. electric vehicle market share as of early 2024.

In [5]:
# Second question — depends on previous context
# Follow-up question that refers to the previous answer
q2 = "How does that compare to last year?"

# Display the follow-up question
print_markdown(f"\nYou: '{q2}'")

# Run the agent again — since there’s no memory, it does not recall the first question/answer
resp2 = await Runner.run(starting_agent = market_researcher_agent, input = q2)

# Display the agent’s response (will fail to connect it to the first question)
print_markdown(f"🤖 Agent:\n{resp2.final_output}")


You: 'How does that compare to last year?'

🤖 Agent:
❌ UNKNOWN: You haven't provided specific data or context about what "that" refers to for comparison with last year.

In [7]:
# Create a session instance
# SQLite-based implementation of session storage.
# This implementation stores conversation history in a SQLite database. 

session = SQLiteSession("conversation")

# First interaction
user_input1 = "What is the market share of Tesla in the US EV market?"
response1 = await Runner.run(
    starting_agent = market_researcher_agent,
    input = user_input1,
    session = session,
)

print_markdown(f"🤖 Agent:\n{response1.final_output}")

🤖 Agent:
✅ FACT: Tesla holds about 60-70% of the US electric vehicle (EV) market share as of early 2024, making it the dominant EV manufacturer in the country.

In [8]:
# Second interaction with history
user_input2 = "How does that compare to last year?"

response2 = await Runner.run(
    starting_agent = market_researcher_agent,
    input = user_input2,
    session = session,
) 

print_markdown(f"🤖 Agent:\n{response2.final_output}")

🤖 Agent:
✅ FACT: Tesla's US EV market share has slightly decreased compared to last year, when it was around 70-75%, due to growing competition from other automakers.